In [199]:
import numpy as np
import matplotlib.pyplot as plt
import galois
import itertools

In [177]:
GF = galois.GF(2)
def find_distance(H):
    # good thing these codes are small
    n = H.shape[1]
    min_weight = n
    for i in range(2**n):
        cw = bin(i)[2:].zfill(n)
        cw = [int(digit) for digit in cw]
        if not np.any((H @ cw) % 2):
            weight = np.count_nonzero(cw)
            if 0 < weight < min_weight:
                min_weight = weight
    return min_weight

def repetition_code(n):
    H = np.zeros((n-1,n))
    for i in range(n-1):
        H[i][i] = H[i][i+1] = 1
    return

def concatenate(H, rep_size):
    # rep = repetition_code(rep_size)
    n = H.shape[1]*rep_size
    k = H.shape[1]-np.linalg.matrix_rank(H)
    new_H = np.zeros((n-k,n), dtype=np.uint8)

    new_H[0:H.shape[0], 0:H.shape[1]] = H
    for i in range(H.shape[1]):
        for j in range(rep_size-1):
            new_H[H.shape[0]+(i*(rep_size-1)+j)][i+(j*H.shape[1])] = 1
            new_H[H.shape[0]+(i*(rep_size-1)+j)][i+((j+1)*H.shape[1])] = 1

    return new_H

In [260]:
def find_best_1D_embedding(H, minimize_checks=False):
    # find best ordering of bit and checks to reduce lr edges
    # good thing the codes are small
    n = H.shape[1]
    m = H.shape[0]

    check_nbhd = []
    for check in range(m):
        bits = np.where(H[check])[1]
        check_nbhd.append(bits)

    best_per = None
    min_val = np.count_nonzero(H)
    best_lr_checks = None
    for per in itertools.permutations(range(n+m)):
        if per <= per[::-1]:
            num_lr = 0
            lr_checks = set()
            bits = np.zeros(n)
            
            counter = 0
            for bit in range(n):
                bits[bit] = per[counter]
                counter += 1

            for check in range(m):
                check_pos = per[counter]

                for bit in check_nbhd[check]:
                    if abs(check_pos-bits[bit]) != 1:
                        num_lr += 1
                        lr_checks |= {check}
                        if minimize_checks:
                            break
                counter += 1

            if num_lr < min_val:
                best_per = per
                min_val = num_lr
                best_lr_checks = lr_checks
            
    return best_per, min_val, best_lr_checks

In [217]:
def search_codes(n, k, d, num_iters):
    best_code = None
    best_weight = n*(n-k)

    for i in range(num_iters):
        H = np.random.randint(2, size=(n-k,n))
        rank = np.linalg.matrix_rank(GF(H))
        if (rank != n-k): continue

        distance = find_distance(H)
        if distance == d:
            if (np.count_nonzero(H) < best_weight):
                best_code = H
                best_distance = distance
    
    return np.matrix(best_code)



def search_codes_dvdc(n, dv, dc):
    # dv, w_c. Every bit is in this many checks
    # dc, w_r. Every check has this many bits in it
    m = (n*dv)//dc
    k = n - m

    vs = np.array([[j for i in range(dv)] for j in range(n)]).flatten()
    cs = np.array([[j for i in range(dc)] for j in range(m)]).flatten()

    H = np.zeros((m, n), dtype=np.uint8)

    while (vs.size and cs.size):
        # choose random 'stub' from each array
        double_edge = True
        while(double_edge):
            v_ind = np.random.randint(0, len(vs))
            c_ind = np.random.randint(0, len(cs))

            if (H[cs[c_ind]][vs[v_ind]] != 1):
                double_edge = False
                H[cs[c_ind]][vs[v_ind]] = 1
                vs = np.delete(vs, v_ind)
                cs =np.delete(cs, c_ind)
        
    print(f"[[{n},{np.linalg.matrix_rank(GF(H))},{find_distance(H)}]]")

In [277]:
# oH = np.matrix([1,1,1])
oH = search_codes(6,2,4,1000)
H = concatenate(oH, 2)

n = H.shape[1]
m = np.linalg.matrix_rank(GF(H))
d = find_distance(H)
print(f"[[{n},{n-m},{d}]]")

[[12,2,8]]


In [286]:
best_emb = find_best_1D_embedding(oH, False)
print(best_emb)

((0, 3, 2, 5, 7, 9, 4, 1, 8, 6), 4, {0, 2, 3})


In [287]:
tot_edges = 2 * np.count_nonzero(H) * (H.shape[0]+H.shape[1])
lr_edges = 2 * np.count_nonzero(H[:oH.shape[0]]) * (H.shape[0]+H.shape[1])
lr_edges_emb = 2 * best_emb[1]  * (H.shape[0]+H.shape[1])

print(f"total edges: {tot_edges}")
print(f"lr edges: {lr_edges}, {np.round(lr_edges/tot_edges, 2)}")
print(f"lr edges: {lr_edges_emb}, {np.round(lr_edges_emb/tot_edges, 2)}")

total edges: 1056
lr edges: 528, 0.5
lr edges: 176, 0.17


In [280]:
find_best_1D_embedding(oH, True)

((0, 3, 2, 4, 5, 6, 7, 1, 8, 9), 3, {0, 2, 3})

In [283]:
hx1 = np.kron(H, np.eye(H.shape[1], dtype=bool))
hx2 = np.kron(np.eye(H.shape[0], dtype=bool), H.T)
Hx = np.hstack([hx1, hx2])

hz1 = np.kron(np.eye(H.shape[1], dtype=bool), H)
hz2 = np.kron(H.T, np.eye(H.shape[0], dtype=bool))
Hz = np.hstack([hz1, hz2])

n = Hx.shape[1]
m = 2*Hx.shape[0]
k = n-m
print(f"[[{n},{k},{d}]]")

[[244,4,8]]


In [285]:
(3*2*10)/m

0.25